In [46]:
from pandas import read_csv, concat, DataFrame
from sklearn.cluster import KMeans

In [2]:
Y = read_csv("dataset/actual.csv")["cancer"]

train = read_csv("dataset/data_set_ALL_AML_train.csv")
test = read_csv("dataset/data_set_ALL_AML_independent.csv")

In [3]:
train_col_list, test_col_list = [str(x) for x in range(1, 39)], [
    str(x) for x in range(39, 73)]

X_train = train[train_col_list].T
X_test = test[test_col_list].T

In [13]:
X = concat([X_train, X_test], ignore_index=True)

In [43]:
clustering = KMeans(n_clusters=2, random_state=0)

clustering.fit(X)

KMeans(n_clusters=2, random_state=0)

In [44]:
Y_pred = clustering.predict(X)

corr = 0

for i in range(len(Y)):
    if Y[i] == "ALL" and Y_pred[i] == 0:
        corr += 1
    elif Y[i] == "AML" and Y_pred[i] == 1:
        corr += 1

purity = corr/len(Y)

print(purity)

0.5694444444444444


In [45]:
print(Y)
print(Y_pred)

0     ALL
1     ALL
2     ALL
3     ALL
4     ALL
     ... 
67    ALL
68    ALL
69    ALL
70    ALL
71    ALL
Name: cancer, Length: 72, dtype: object
[0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 0 1 0 1 0 1 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0]


In [48]:
clus_0_ALL, clus_0_AML = 0, 0
clus_1_ALL, clus_1_AML = 0, 0

for i in range(len(Y)):
    if Y_pred[i] == 0:
        if Y[i] == "ALL":
            clus_0_ALL += 1
        else:
            clus_0_AML += 1
    else:
        if Y[i] == "ALL":
            clus_1_ALL += 1
        else:
            clus_1_AML += 1

print(f"Cluster 0 - ALL : {clus_0_ALL} , AML : {clus_0_AML}")
print(f"Cluster 1 - ALL : {clus_1_ALL} , AML : {clus_1_AML}")

Cluster 0 - ALL : 33 , AML : 17
Cluster 1 - ALL : 14 , AML : 8


In [49]:
from sklearn.metrics import confusion_matrix

# mapping = {0: "ALL", 1: "AML"}
mapping={0:"ALL", 1:"AML"} if clus_0_ALL>clus_0_AML else {0:"AML",1:"ALL"}

Y_pred_converted = list(map(lambda val: mapping[val], Y_pred))

contingency = confusion_matrix(Y, Y_pred_converted)

contingency_df = DataFrame(contingency, index=["ALL", "AML"], columns=[
                           "Cluster 0", "Cluster 1"])

print(contingency_df)

     Cluster 0  Cluster 1
ALL         33         14
AML         17          8


In [51]:
clus_0_purity = contingency_df["Cluster 0"].max(
)/contingency_df["Cluster 0"].sum()
clus_1_purity = contingency_df["Cluster 1"].max(
)/contingency_df["Cluster 1"].sum()

print(f"Cluster 0 - Purity : {clus_0_purity}")
print(f"Cluster 1 - Purity : {clus_1_purity}")

purity = (clus_0_purity*contingency_df["Cluster 0"].sum()+clus_1_purity*contingency_df["Cluster 1"].sum())/(
    contingency_df["Cluster 0"].sum()+contingency_df["Cluster 1"].sum())

print(f"Purity : {purity}")

Cluster 0 - Purity : 0.66
Cluster 1 - Purity : 0.6363636363636364
Purity : 0.6527777777777778
